## Check GPU Availability

In [1]:
!nvidia-smi

Tue Nov 14 16:48:06 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    On  | 00000000:00:1E.0 Off |                    0 |
|  0%   19C    P8              15W / 300W |      2MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [29]:
import os
cache_dir = os.makedirs("cache3",exist_ok=True)
os.environ['TRANSFORMERS_CACHE'] = "cache3"
os.environ['HF_DATASETS_CACHE'] = "cache3"

## Install required libraries

In [30]:
!pip3 install trl transformers accelerate git+https://github.com/huggingface/peft.git -Uqqq
!pip3 install datasets bitsandbytes einops wandb -Uqqq

## Importing libraries

In [31]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings
warnings.filterwarnings("ignore")

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [28]:
import pandas as pd
df =pd.read_csv("FINAL_V4_MORE_EPOCHS_EQUAL.csv",on_bad_lines="skip",encoding="ISO-8859-1")

In [29]:
df.shape

(317056, 2)

In [30]:
#42957832

In [31]:
# df

In [32]:
#6897103

# df = df[:6897103]

In [33]:
df

,target_text,input_text
0,"434 100TH AVE SE , , , US , WA , OLYMPIA , ...",SELECT THE ADDRESSES IN CONTEXT WHICH ARE ON S...
1,"424 100TH AVE SE , , , US , WA , OLYMPIA , 9...",SELECT THE ADDRESSES IN CONTEXT WHICH ARE ON S...
2,"424 100TH AVE SE , , , US , WA , OLYMPIA , 9...",SELECT THE ADDRESSES IN CONTEXT WHICH ARE ON S...
3,"424 100TH AVE SE , , , US , WA , OLYMPIA , 9...",SELECT THE ADDRESSES IN CONTEXT WHICH ARE ON S...
4,"424 100TH AVE SE , , , US , WA , OLYMPIA , 9...",SELECT THE ADDRESSES IN CONTEXT WHICH ARE ON S...
...,...,...
317051,Yes,CHECK IF GIVEN TWO ADDRESSES ARE IN SAME NEIGH...
317052,No,CHECK IF GIVEN TWO ADDRESSES ARE IN SAME NEIGH...
317053,No,CHECK IF GIVEN TWO ADDRESSES ARE IN SAME NEIGH...
317054,Yes,CHECK IF GIVEN TWO ADDRESSES ARE IN SAME NEIGH...


In [34]:
print(df['input_text'][0])

SELECT THE ADDRESSES IN CONTEXT WHICH ARE ON SAME STREET AS OF THE GIVEN ADDRESS : 

GIVEN ADDRESS : 424 100TH AVE SE ,  ,  , US , WA , OLYMPIA , 98501-9710 

CONTEXT : 
 608 W EMERSON ST APT 321 ,  ,  , US , WA , SEATTLE , 98119-1544   |   805 W EMERSON ST ,  ,  , US , WA , SEATTLE , 98119-1457  |  434 100TH AVE SE ,  ,  , US , WA , OLYMPIA , 98501 


In [35]:
df.rename(columns = {'input_text':'<HUMAN>: '}, inplace = True)
df.rename(columns = {'target_text':'<ASSISTANT>: '}, inplace = True)

In [36]:
# df.head(2)

In [37]:
df['<HUMAN>: '] = '<HUMAN>: ' +df['<HUMAN>: '].astype(str)
df['<ASSISTANT>: '] = '<ASSISTANT>: ' +df['<ASSISTANT>: '].astype(str)

In [38]:
df.head(5)

,<ASSISTANT>:,<HUMAN>:
0,"<ASSISTANT>: 434 100TH AVE SE , , , US , WA...",<HUMAN>: SELECT THE ADDRESSES IN CONTEXT WHICH...
1,"<ASSISTANT>: 424 100TH AVE SE , , , US , WA ...",<HUMAN>: SELECT THE ADDRESSES IN CONTEXT WHICH...
2,"<ASSISTANT>: 424 100TH AVE SE , , , US , WA ...",<HUMAN>: SELECT THE ADDRESSES IN CONTEXT WHICH...
3,"<ASSISTANT>: 424 100TH AVE SE , , , US , WA ...",<HUMAN>: SELECT THE ADDRESSES IN CONTEXT WHICH...
4,"<ASSISTANT>: 424 100TH AVE SE , , , US , WA ...",<HUMAN>: SELECT THE ADDRESSES IN CONTEXT WHICH...


In [39]:
df['text'] = df['<HUMAN>: '] + '\n' + df['<ASSISTANT>: ']

In [40]:
df = df.drop(['<ASSISTANT>: ','<HUMAN>: '],axis=1)

In [41]:
print(df['text'][0])

<HUMAN>: SELECT THE ADDRESSES IN CONTEXT WHICH ARE ON SAME STREET AS OF THE GIVEN ADDRESS : 

GIVEN ADDRESS : 424 100TH AVE SE ,  ,  , US , WA , OLYMPIA , 98501-9710 

CONTEXT : 
 608 W EMERSON ST APT 321 ,  ,  , US , WA , SEATTLE , 98119-1544   |   805 W EMERSON ST ,  ,  , US , WA , SEATTLE , 98119-1457  |  434 100TH AVE SE ,  ,  , US , WA , OLYMPIA , 98501 
<ASSISTANT>:  434 100TH AVE SE ,  ,  , US , WA , OLYMPIA , 98501 


In [42]:
df.to_csv("FINAL_FALCON_MORE_EPOCHS_EQUAL_FINAL.csv",index=False)

In [25]:
from datasets import Dataset
Dataset.cleanup_cache_files
import pandas as pd
df =pd.read_csv("FINAL_V4_MORE_EPOCHS_EQUAL.csv",on_bad_lines="skip",encoding="ISO-8859-1")
import transformers
from datasets import load_dataset, load_metric
medium_datasets = Dataset.from_pandas(df)
train_testvalid = medium_datasets.train_test_split(test_size=0.001, shuffle=True)

test_valid = train_testvalid["test"].train_test_split(test_size=0.9, shuffle=True)

import datasets
medium_datasets = datasets.DatasetDict({
"train": train_testvalid["train"],
"validation": test_valid["train"],
"test": test_valid["test"]})

In [26]:
medium_datasets

DatasetDict({
    train: Dataset({
        features: ['target_text', 'input_text'],
        num_rows: 316738
    })
    validation: Dataset({
        features: ['target_text', 'input_text'],
        num_rows: 31
    })
    test: Dataset({
        features: ['target_text', 'input_text'],
        num_rows: 287
    })
})

In [27]:
medium_datasets['train'][1]

{'target_text': '6830 PRAIRIE RIDGE DR NE ,  ,  , US , WA , OLYMPIA , 98516-1137  |  6830 PRAIRIE RIDGE DR NE ,  ,  , US , WA , OLYMPIA , 98516-1137 ',
 'input_text': 'SELECT THE ADDRESSES IN CONTEXT WHICH ARE ON SAME NEIGHBOURS AS OF THE GIVEN ADDRESS : \n\nGIVEN ADDRESS :  6830 PRAIRIE RIDGE DR NE ,  ,  , US , WA , OLYMPIA , 98516 \n\nCONTEXT : \n6830 PRAIRIE RIDGE DR NE ,  ,  , US , WA , OLYMPIA , 98516-1137   |  5422 KOMACHIN LOOP SE ,  ,  , US , WA , LACEY , 98513  |   6830 PRAIRIE RIDGE DR NE ,  ,  , US , WA , OLYMPIA , 98516-1137 '}

## Load custom Mental Health conv dataset

In [23]:
#data = load_dataset("heliosbrahma/mental_health_chatbot_dataset",cache_dir='cache3')
data = medium_datasets

In [24]:
data

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 6890205
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 6829
    })
    test: Dataset({
        features: ['text'],
        num_rows: 69
    })
})

In [25]:
print(data["train"][1])

{'text': '<HUMAN>: FIND STREET IN GIVEN ADDRESS : \n\nGIVEN ADDRESS : AMERICAN PARKINSON DISEASE ASSOCIATION , 180 NICKERSON ST STE 108 ,  , US , WA , SEATTLE , 98109-1631\n<ASSISTANT>: NICKERSON_ST'}


In [26]:
print(data["train"][1]['text'])

<HUMAN>: FIND STREET IN GIVEN ADDRESS : 

GIVEN ADDRESS : AMERICAN PARKINSON DISEASE ASSOCIATION , 180 NICKERSON ST STE 108 ,  , US , WA , SEATTLE , 98109-1631
<ASSISTANT>: NICKERSON_ST


In [27]:
#print(data["train"][1]['text'])

## Model Training

In [45]:
model_name = "ybelkada/falcon-7b-sharded-bf16" # sharded falcon-7b model

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,            # load model in 4-bit precision
    bnb_4bit_quant_type="nf4",    # pre-trained model should be quantized in 4-bit NF format
    bnb_4bit_use_double_quant=True, # Using double quantization as mentioned in QLoRA paper
    bnb_4bit_compute_dtype=torch.bfloat16, # During computation, pre-trained model should be loaded in BF16 format
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config, # Use bitsandbytes config
    device_map="auto",  # Specifying device_map="auto" so that HF Accelerate will determine which GPU to put each layer of the model on
    trust_remote_code=True, # Set trust_remote_code=True to use falcon-7b model with custom code
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [46]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,cache_dir='cache3') # Set trust_remote_code=True
tokenizer.pad_token = tokenizer.eos_token # Setting pad_token same as eos_token

In [47]:
model = prepare_model_for_kbit_training(model)

lora_alpha = 32 # scaling factor for the weight matrices
lora_dropout = 0.05 # dropout probability of the LoRA layers
lora_rank = 4 # dimension of the low-rank matrices

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",  # setting to 'none' for only training weight params instead of biases
    task_type="CAUSAL_LM",
    target_modules=[         # Setting names of modules in falcon-7b model that we want to apply LoRA to
        "query_key_value",
#         "dense",
#         "dense_h_to_4h",
#         "dense_4h_to_h",
    ]
)

peft_model = get_peft_model(model, peft_config)

In [51]:
output_dir = "./falcon-7b-SFT"
per_device_train_batch_size = 16 # reduce batch size by 2x if out-of-memory error
gradient_accumulation_steps = 4  # increase gradient accumulation steps by 2x if batch size is reduced
optim = "paged_adamw_32bit" # activates the paging for better memory management
save_strategy="steps" # checkpoint save strategy to adopt during training
save_steps = 10 # number of updates steps before two checkpoint saves
logging_steps = 10  # number of update steps between two logs if logging_strategy="steps"
learning_rate = 2e-4  # learning rate for AdamW optimizer
max_grad_norm = 0.3 # maximum gradient norm (for gradient clipping)
#max_steps = 320        # training will happen for 320 steps
warmup_ratio = 0.03 # number of steps used for a linear warmup from 0 to learning_rate
lr_scheduler_type = "cosine"  # learning rate scheduler

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=2,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    #bf16=True,
    max_grad_norm=max_grad_norm,
    #max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=False,
    report_to="none",
    bf16=torch.cuda.get_device_capability(torch.cuda.current_device())[0] >= 8,
    #model=model.to('cpu'),
)

In [52]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=256,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/6890205 [00:00<?, ? examples/s]

In [53]:
# upcasting the layer norms in torch.bfloat16 for more stable training
# for name, module in trainer.model.named_modules():
#     if "norm" in name:
#         module = module.to(torch.bfloat16)

In [54]:
# authenticate WandB for logging metrics
# import wandb
# wandb.login()

In [55]:
peft_model.config.use_cache = False
trainer.train()

Step,Training Loss
10,1.603400
20,0.958700
30,0.672500


KeyboardInterrupt: 

In [ ]:
peft_model.save_pretrained("output_dir")

In [18]:
#trainer.push_to_hub()

## Inference Pipeline

In [1]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings
warnings.filterwarnings("ignore")

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [2]:
# Loading original model
model_name = "ybelkada/falcon-7b-sharded-bf16"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    load_in_8bit=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
# Loading PEFT model
PEFT_MODEL = "output_dir"

config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [6]:
# Function to generate responses from both original model and PEFT model and compare their answers.
def generate_answer(query):
  system_prompt = """Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'."""

  user_prompt = f"""<HUMAN>: {query}
  <ASSISTANT>: """

  final_prompt = system_prompt + "\n" + user_prompt

  device = "cuda:0"
  dashline = "-".join("" for i in range(50))

  encoding = tokenizer(final_prompt, return_tensors="pt").to(device)
  outputs = model.generate(input_ids=encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = tokenizer.eos_token_id, \
                                                                                                                     eos_token_id = tokenizer.eos_token_id, attention_mask = encoding.attention_mask, \
                                                                                                                     temperature=0.4, top_p=0.6, repetition_penalty=1.3, num_return_sequences=1,))
  text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

  print(dashline)
  print(f'ORIGINAL MODEL RESPONSE:\n{text_output}')
  print(dashline)

  peft_encoding = peft_tokenizer(final_prompt, return_tensors="pt").to(device)
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=56, pad_token_id = peft_tokenizer.eos_token_id, \
                                                                                                                     eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                     temperature=0.4, top_p=0.6, repetition_penalty=1.3, num_return_sequences=1,))
  peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

  print(f'PEFT MODEL RESPONSE:\n{peft_text_output}')
  print(dashline)

## Compare responses between Original model and PEFT model

In [25]:
query = "How can I prevent anxiety and depression?"
generate_answer(query)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...

-------------------------------------------------
ORIGINAL MODEL RESPONSE:
Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'.
<HUMAN>: How can I prevent anxiety and depression?
  <ASSISTANT>: 'Anxiety and depression are caused by a chemical imbalance in the brain. The brain is made up of two parts, the left and right hemisphere. The left hemisphere is responsible for logical thinking, while the right hemisphere is responsible for creativity. When the two hemispheres are not in balance, it can lead to anxiety and depression. There are many ways to prevent anxiety and depression, but the most important thing is to maintain a healthy lifestyle. This includes eating a balanced diet, getting enough sleep, exercising regularly, and avoiding drugs and alcohol. If you are experiencing symptoms of anxiety or depressio

In [7]:
query = "FIND STREET IN GIVEN ADDRESS : \n\nGIVEN ADDRESS : AMERICAN PARKINSON DISEASE ASSOCIATION , 180 NICKERSON ST STE 108 ,  , US , WA , SEATTLE , 98109-1631\n"
generate_answer(query)      
        

-------------------------------------------------
ORIGINAL MODEL RESPONSE:
Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'.
<HUMAN>: FIND STREET IN GIVEN ADDRESS : 

GIVEN ADDRESS : AMERICAN PARKINSON DISEASE ASSOCIATION, 180 NICKERSON ST STE 108, , US, WA, SEATTLE, 98109-1631

  <ASSISTANT>: 'AMERICAN PARKINSON DISEASE ASSOCIATION', '180 NICKERSON ST STE 108', 'SEATTLE', 'WA', '98109-1631', 'US', 'SEATTLE', 'WA', 'SEATTLE', 'WA', 'SEATTLE', 'WA', 'SEATTLE', 'WA', 'SEATTLE', 'WA', 'SEATTLE', 'WA', 'SEATTLE', 'WA', 'SEATTLE', 'WA', 'SEATTLE', 'WA', 'SEATTLE', 'WA', 'SEATTLE', 'WA', 'SEATTLE', 'WA', 'SEATTLE', 'WA', 'SEATTLE', 'WA' 
-------------------------------------------------
PEFT MODEL RESPONSE:
Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the a